# Setup

In [1]:
#Analise Descritiva 
import pandas as pd
import numpy as np

#Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact



from sqlalchemy import create_engine, inspect, text
import os

In [2]:
engine = create_engine(os.environ["REDSHIFT_READ_URI"], echo=False)

def sql_df(query):
  with engine.connect() as conexao:
    consulta = conexao.execute(text(query))
    dados = consulta.fetchall()
  df = pd.DataFrame(dados, columns= consulta.keys())
  return df

In [3]:
pd.set_option('display.float_format', lambda x: '{:.0f}'.format(x))

# Bases

In [12]:
com = pd.read_fwf("C:\\Users\\mateus.schualtz\\Downloads\\REM_CONTRATOS_WHITELIST_20240807_032359.txt", encoding='latin1')
com.rename(columns={'REMESSA_CONTRATOS_ASSESSORIA':'BASE'}, inplace=True)
com = com[['BASE']]

In [13]:
query = ''' 

select 
document
from quiteja_lake.deal d 
left join quiteja_lake_restrict.customer c on d.customer_id = c.id 
where creditor_id = 39 and d.last_import_at::date = '2024-08-07'
'''

In [14]:
importacao = sql_df(query)
importacao.rename(columns={'document':'Importados'}, inplace=True)

In [15]:
consolidacao = pd.read_fwf("C:\\Users\\mateus.schualtz\\Downloads\\REM_CONTRATOS_WHITELIST_20240808_032407.txt", encoding='latin1')
consolidacao.rename(columns={'REMESSA_CONTRATOS_ASSESSORIA':'BASE'}, inplace=True)
consolidacao = consolidacao[['BASE']]

In [16]:
qcon = ''' 

select 
customer_document
from consolidacao_carteira.customer_39 c 

'''

qcon = sql_df(qcon)
qcon.rename(columns={'customer_document':'Consolidacao'}, inplace=True)

In [17]:
importacao = com.merge(importacao, left_on='BASE', right_on='Importados', how='left')
consolidacao = consolidacao.merge(qcon, left_on='BASE', right_on='Consolidacao', how='left')

In [18]:
n_importados = pd.DataFrame(importacao.isnull().sum(), columns=['Total Não Importado'])
n_importados['Percentual da Base'] = round((n_importados['Total Não Importado'] / len(com)).astype(float) * 100)

In [21]:
n_consolidados = pd.DataFrame(consolidacao.isnull().sum(), columns=['Total Não Consolidado'])
n_consolidados['Percentual da Base'] = round((n_consolidados['Total Não Consolidado'] / len(qcon)).astype(float) * 100)

In [19]:
n_importados

,Total Não Importado,Percentual da Base
BASE,0,0
Importados,11939,3


In [22]:
n_consolidados

,Total Não Consolidado,Percentual da Base
BASE,0,0
Consolidacao,13226,4
